In [1]:
# use this block if I get scraping problems / get blocked
from scraper_api import ScraperAPIClient
import pandas as pd
import requests
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import math
import yahoo_fin.stock_info as yf

# api_key = '483eb326ba2467782055a36e0ad7f60a'
# client = ScraperAPIClient(api_key)

# page break

#### note for the time frame, you "could" go back up to 4 years aka 1461 days (or maybe more via the custom option) but the serious problem here is that the site only lets you go 9 pages back at most for any given query (where every page has 1000 rows). IOW, for any given set of query criteria, you can get at most the 9,000 top results after making 9 requests to the first 9 pages of 1000 entries each. FOR PRACTICAL TRADING REASONS, WE WILL ONLY BE GOING BACK 30 DAYS AKA A MONTH TO START. Eventually, we will be looking for the % change in holder shares among other things. 

In [2]:
url = "http://www.nasdaqtrader.com/dynamic/symdir/nasdaqlisted.txt"

page = requests.get(url)

tickers = []

for line in page.text.split('\n')[1:-2]:
    tickers.append(line[:line.index('|')])

allStocks = dict.fromkeys(tickers, {})
# allStocks

In [3]:
topPerformers = ["Sr.VP", "Chairman", "VP GC", "Mktg", "Engineering", "Chief People Officer", "Corporate Development", "Business Dev", "Corp Secretary", "Co-Founder", "Global Sales", "Corp. Sec.", "VP Sales", "Interim CEO", "Principal Accounting Officer", "Admin.", "Member of Group > 10%", "Chief Business Officer", "SVP-GC", "CRO", "Retail", "VP Finance", "CSO", "Division Pres", "HR", "Pres of Subsidiary", "EVP - Operations", "Sec", "Gen. Counsel", "Sr. Vice Pres", "Corporate VP", "PAO", "North America", "Exec VP", "Sales", "GM", "C.E.O.", "Corp. Dev.", "Secy.", "Asia", "Chief Scientific Officer", "International", "Marketing", "CMO", "Exec.VP", "Ex. VP", "Chief Operations Officer", "CHRO", "Chief Revenue Officer", "See remarks", "13(d) group member - >10%", "VP of Finance", "SVP Finance", "VP Operations", "Secy", "D", "VP of Marketing", "Exec Officer", "VP Corporate Development", "Acting CFO", "Vice-Pres", "Chief Risk Officer", "VP HR", "Research", "Operations", "VP of Operations", "SVP of Sales", "Controller", "MD", "CFO", "See Remarks", "E.VP", "EVP - CFO", "GC", "VP of Sales", "Chief Admin Officer", "Worldwide Sales", "IT", "Treasurer", "Sec'y", "Strategy", "Chief Development Officer", "VP", "CTO", "Sec.", "Exec COB", "Chief Administrative Officer", "VP - Finance", "Senior Vice-Pres", "SVP", "Chief Accounting Officer", "CIO", "Senior VP", "Sr EVP", "VP-Finance", "EVP Finance", "IR"]


In [4]:
#load in from csv from past few years only (we'll use the Remaining.csv file, which only has data from last 3ish years)

infile = r'~/Documents/AlgorithmicTrading/Insider Trading Web Scraping/excels/sortedAll2004-2021Remaining.csv'

data = pd.read_csv(infile, skiprows = 0)
data = data.drop(['Unnamed: 0'], axis=1)
for index, row in data.iterrows():
    if row['deltOwned'][0]=='N':
        data.loc[index, "deltOwned"] = 0
        data.loc[index, "Value"] = float(row['Price'])*int(row['Qty'])

data['Trade Date'] = pd.to_datetime(data['Trade Date'])
data=data.sort_values(by='Trade Date')
data


/Users/patrick/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Trade Date,Ticker,Insider Name,Company Name,Title,Trade Type,Price,Qty,Owned,deltOwned,Value
0,2019-01-15,RMD,Douglas Robert Andrew,Resmed Inc,"Pres, COO ResMed Inc.",S - Sale+OE,114.18,-4188,193753,-2,-478186
25,2019-01-15,TNDM,Hansen Brian B,Tandem Diabetes Care Inc,"EVP, CHIEF COMMERCIAL OFFICER",S - Sale+OE,41.54,-25000,1381,-95,-1038528
26,2019-01-15,PLAY,King Stephen M,"Dave &amp; Buster'S Entertainment, Inc.",Dir,S - Sale+OE,53.17,-15000,32091,-32,-797592
27,2019-01-15,PLAY,Manning Margo Lynn,"Dave &amp; Buster'S Entertainment, Inc.",COO,S - Sale+OE,52.88,-5000,8563,-37,-264400
28,2019-01-15,VEEV,Mateo Alan,Veeva Systems Inc,EVP Global Sales,S - Sale+OE,97.61,-622,25757,-2,-60713
...,...,...,...,...,...,...,...,...,...,...,...
191476,2022-04-21,CLNN,General Resonance LLC,Clene Inc.,10%,S - Sale,3.01,-132891,15742030,-1,-400002
191475,2022-04-21,PGR,Mascaro Daniel P,Progressive Corp/oh/,"Vice Pres, Secretary, CLO",S - Sale,113.59,-558,39445,-1,-63383
191492,2022-04-21,AWI,Hershey Mark A,Armstrong World Industries Inc,SVP Americas,S - Sale+OE,95.11,-1365,45053,-3,-129825
191483,2022-04-21,MLI,Hansen John B,Mueller Industries Inc,Dir,S - Sale,60.33,-2000,70885,-3,-120660


In [5]:
#filter so the newdf only contains trades made by top performer titles
newdf = pd.DataFrame(columns=data.columns)

for index, row in data.iterrows():
    
    try:
        rowTitles = row['Title'].split(', ')
    except:
        continue
    
    for pos in rowTitles:
        if pos in topPerformers:
            newdf.loc[len(newdf)] = row
            break
newdf

,Trade Date,Ticker,Insider Name,Company Name,Title,Trade Type,Price,Qty,Owned,deltOwned,Value
0,2019-01-15,RDFN,Nielsen Christopher John,Redfin Corp,CFO,S - Sale+OE,17.06,-3000,0,-100,-51167
1,2019-01-15,SITE,Convers Pascal,"Siteone Landscape Supply, Inc.","EVP, Strategy, Development",S - Sale+OE,52.75,-9750,0,-100,-514313
2,2019-01-15,YUM,Gibbs David W,Yum Brands Inc,"Pres, CFO",S - Sale+OE,89.93,-6410,39346,-14,-576451
3,2019-01-15,CIEN,Alexander Stephen B,Ciena Corp,"SVP, CTO",S - Sale,36.79,-2500,123395,-2,-91986
4,2019-01-15,BJ,Luce Graham,"Bj'S Wholesale Club Holdings, Inc.","SVP, Secretary",S - Sale+OE,23.50,-10000,17577,-36,-235000
...,...,...,...,...,...,...,...,...,...,...,...
55133,2022-04-20,ALVR,Atillasoy Ercem,"Allovir, Inc.",See Remarks,S - Sale,6.04,-2040,121563,-2,-12329
55134,2022-04-21,NTGR,Cormack Heidi,"Netgear, Inc.","SVP, Marketing",S - Sale+OE,22.52,-2105,61421,-3,-47405
55135,2022-04-21,GLSI,Patel Snehal,"Greenwich Lifesciences, Inc.","CEO, CFO, 10%",P - Purchase,16.09,6000,1216929,0,96540
55136,2022-04-21,KR,Massa Timothy A,Kroger Co,SVP,S - Sale+OE,58.03,-50283,163956,-23,-2917774


In [14]:
# get purchases only only
purchaseDF = newdf.loc[newdf['Trade Type'] == 'P - Purchase']
# newdf
purchaseDF

,Trade Date,Ticker,Insider Name,Company Name,Title,Trade Type,Price,Qty,Owned,deltOwned,Value
20,2019-01-16,EDUC,O Keefe Daniel E,Educational Development Corp,CFO,P - Purchase,7.63,1299,8372,18,9911
21,2019-01-16,CMC,Lindsey Mary A,Commercial Metals Co,"SVP, CFO",P - Purchase,15.92,1000,145089,1,15920
55,2019-01-17,RBKB,Dwyer Francis X.,"Rhinebeck Bancorp, Inc.",Pres of Subsidiary,P - Purchase,11.71,250,8385,3,2928
56,2019-01-17,EDUC,O Keefe Daniel E,Educational Development Corp,CFO,P - Purchase,7.75,701,9073,8,5433
57,2019-01-17,TLRS,McDowell Donald James,Timberline Resources Corp,VP Corporate Development,P - Purchase,0.09,4000,163700,3,376
...,...,...,...,...,...,...,...,...,...,...,...
55108,2022-04-20,STSS,Crescenzo Andrew R.,Sharps Technology Inc.,CFO,P - Purchase,2.26,6000,6000,0,13560.0
55109,2022-04-20,FRAF,Hollar Mark R,Franklin Financial Services Corp /pa/,CFO,P - Purchase,32.99,3,7767,0,99
55111,2022-04-20,FRAF,Carmack Karen K,Franklin Financial Services Corp /pa/,SVP,P - Purchase,32.99,1,2226,0,33
55116,2022-04-20,CRTD,Frommer Jeremy,"Creatd, Inc.",Exec COB,P - Purchase,1.07,1026,644709,0,1098


# separator

#### instead of running the next couple blocks (which take a while), just read in the DF from csv ( 'topPerformingInsiderStocks2019-2022.csv' )

In [79]:
from datetime import datetime

def getAnnualEPS5Years(ticker):
    isOldStock = True
    hist = yf.get_earnings_history(ticker)
    currentDate = datetime.today().strftime('%Y-%m-%d')
    recentEarnings = []
    annualEPSLast5Years = []
    avgSurprise = 0
    surprises = []
    
    #if the company ipo'd 
    if len(hist)<20:
        
        for ind, earnings in enumerate(hist):
            if earnings['startdatetime'][:10] < currentDate:
                #we are at the most recent earnings
                recentEarnings = hist[ind:len(hist)]

                isOldStock = False
                for i in range(len(hist)//4):
                    yearSum = 0
                    for j in range(4):
                        spot = i*4 + j
                        surprises.append(hist[ind+spot]['epssurprisepct'])
                        yearSum += recentEarnings[spot]['epsactual']
                    annualEPSLast5Years.append(yearSum)
                annualEPSLast5Years.reverse()
                return annualEPSLast5Years, surprises, isOldStock
    
    
    
    #if it is a recent IPO, then it will shoot an error. in the "main" where this function is called, we will thus skip recent-IPO stocks with try/except
    for ind, earnings in enumerate(hist):
        if earnings['startdatetime'][:10] < currentDate:
            #we are at the most recent earnings
            recentEarnings = hist[ind:ind+20]
            for i in range(5):
                yearSum = 0
                for j in range(4):
                    spot = i*4 + j
                    
                    surprises.append(hist[ind+spot]['epssurprisepct'])
                    
                    yearSum += recentEarnings[spot]['epsactual']
                annualEPSLast5Years.append(yearSum)
            annualEPSLast5Years.reverse()
            return annualEPSLast5Years, surprises, isOldStock
            break
# print(annualEPSLast5Years)

def getIncEpsScore(yearlyEarnings):
    incCount = 0
    for year in range(len(yearlyEarnings)):
        if year == 0:
            continue
        if yearlyEarnings[year] > yearlyEarnings[year-1]:
            incCount+=1
    return incCount


# getAnnualEPS5Years('ee')


In [78]:
yf.get_earnings_history('msft')

[{'ticker': 'MSFT',
  'companyshortname': 'Microsoft Corp',
  'startdatetime': '2023-01-23T10:59:00.000Z',
  'startdatetimetype': 'TNS',
  'epsestimate': None,
  'epsactual': None,
  'epssurprisepct': None,
  'timeZoneShortName': 'EST',
  'gmtOffsetMilliSeconds': -18000000,
  'quoteType': 'EQUITY'},
 {'ticker': 'MSFT',
  'companyshortname': 'Microsoft Corp',
  'startdatetime': '2022-10-24T10:59:00.000Z',
  'startdatetimetype': 'TNS',
  'epsestimate': None,
  'epsactual': None,
  'epssurprisepct': None,
  'timeZoneShortName': 'EDT',
  'gmtOffsetMilliSeconds': -14400000,
  'quoteType': 'EQUITY'},
 {'ticker': 'MSFT',
  'companyshortname': 'Microsoft Corp',
  'startdatetime': '2022-07-25T10:59:00.000Z',
  'startdatetimetype': 'TNS',
  'epsestimate': None,
  'epsactual': None,
  'epssurprisepct': None,
  'timeZoneShortName': 'EDT',
  'gmtOffsetMilliSeconds': -14400000,
  'quoteType': 'EQUITY'},
 {'ticker': 'MSFT',
  'companyshortname': 'Microsoft Corp',
  'startdatetime': '2022-04-26T20:00:

In [90]:
# below 2 lines commented in order to resume in car from middle
rankedDF = pd.DataFrame(columns=['Ticker', 'Price', 'Projected Price Growth', 'Market Cap', 'PEG', 'PEG Rank', 'PE', 'PE Rank', '5 Year EPS Growth', 'EPS Growth Rank', 'Increasing EPS Score 0-4', 'EPS Surprise Avg', 'ROE', 'ROE Rank', 'Operating Cash Flow Ratio', 'Operating Cash Flow Ratio Rank', 'Current Ratio', 'Current Ratio Rank', 'Total Avg Rank', 'Total Avg Percentile'])
listedTickers = list(set(purchaseDF['Ticker']))

# DF that scores stocks on exceeding threshholds for each metric and also ranks them relatively for each metric and all metrics averaged
# metrics are PEG (fwd?), quick ratio, ROIC, Debt/Equity, PE, 5-year EPS growth %, 
# do something to see company's relative price right now and if it is trading at a discount

# include market cap??? can do that later
# would like to do roic, but harder to get/set up data, so we'll do ROE for now

skips = 0
skipped = [] #we'll just come back to the skipped stocks and review them later without trying to fix lack analyst forecasts (which is causing the errors)

for tickNum, ticker in enumerate(listedTickers):
    print('------------------\n', tickNum, ticker)
    try:
        result = yf.get_quote_data(ticker)
        PE = result['priceEpsCurrentYear']
    #     fwdPE = result['forwardPE']
    #     trailingPE = result['trailingPE']  #using PE instead of trailing PE
        futureEpsGrowth = (result['epsForward'] - result['epsCurrentYear'])*100/result['epsForward']
    #     prevEpsGrowth = (result['epsCurrentYear'] - result['epsTrailingTwelveMonths'])*100/result['epsTrailingTwelveMonths']
        futurePeg = PE/futureEpsGrowth
    #     currentPeg = trailingPE/prevEpsGrowth
        marketCap = result['marketCap']
        
        quoteTable = yf.get_quote_table(ticker)
        price = round(quoteTable['Quote Price'], 2)
#         PE = quoteTable['PE Ratio (TTM)']
        oneYearTarget = quoteTable['1y Target Est']
#         print('here')
        projectedPriceChangePerc = (oneYearTarget - price)*100/price
        
        print(price, oneYearTarget)
        
        eps5Years, surprises, oldStock = getAnnualEPS5Years(ticker)
        epsGrowth5Years = (eps5Years[-1]-eps5Years[0])*100/eps5Years[0]
        increasingEpsScore = getIncEpsScore(eps5Years)
        avgSurprise = sum(surprises)/len(surprises)

        ROE = float(yf.get_stats(ticker).loc[34]['Value'][:-1]) # a percent

        financials = yf.get_financials(ticker, yearly=True)
        operatingCashFlow = float(format(financials['yearly_cash_flow'].loc['totalCashFromOperatingActivities'][0], 'f')) / float(format(financials['yearly_balance_sheet'].loc['totalLiab'][0], 'f'))
        currentRatio = float(format(financials['yearly_balance_sheet'].loc['totalCurrentAssets'][0], 'f')) / float(format(financials['yearly_balance_sheet'].loc['totalCurrentLiabilities'][0], 'f'))

        print('Ticker', ticker, 'Price', price, 'Projected Price Growth', projectedPriceChangePerc, 'Market Cap', marketCap, 'PEG', futurePeg, 'PEG Rank', 'PE', PE, 'PE Rank', '5 Year EPS Growth', epsGrowth5Years, 'EPS Growth Rank', 'Increasing EPS Score 0-4', increasingEpsScore, 'EPS Surprise Avg', avgSurprise, 'ROE', ROE, 'ROE Rank', 'Operating Cash Flow', operatingCashFlow, 'Operating Cash Flow Rank', 'Current Ratio', currentRatio, 'Current Ratio Rank', 'Total Avg Rank', 'Total Avg Percentile', sep='\n')
        rankedDF = rankedDF.append({'Ticker':ticker, 
          'Price':price, 
          'Projected Price Growth': projectedPriceChangePerc,
          'Market Cap':marketCap, 
          'PEG':futurePeg, 
#           'PEG Rank':0, 
          'PE':PE, 
#           'PE Rank':0, 
          '5 Year EPS Growth':epsGrowth5Years, 
#           'EPS Growth Rank':0, 
          'Increasing EPS Score 0-4':increasingEpsScore, 
#           'Increasing EPS Rank': 0,
          'EPS Surprise Avg':avgSurprise, 
          'ROE':ROE, 
#           'ROE Rank':0, 
          'Operating Cash Flow Ratio':operatingCashFlow, 
#           'Operating Cash Flow Ratio Rank':0, 
          'Current Ratio':currentRatio, 
#           'Current Ratio Rank':0, 
          'Total Avg Rank':0, 
          'Total Avg Percentile':0 }, ignore_index=True)
        
        print()
        print()
    except:
        skips += 1
        skipped.append(ticker)
        continue

        

------------------
 0 VLY
12.43 15.88
Ticker
VLY
Price
12.43
Projected Price Growth
27.755430410297677
Market Cap
6294403072
PEG
0.8339037211764708
PEG Rank
PE
10.272727
PE Rank
5 Year EPS Growth
75.75757575757578
EPS Growth Rank
Increasing EPS Score 0-4
4
EPS Surprise Avg
-1.1744999999999999
ROE
9.79
ROE Rank
Operating Cash Flow
0.021821953316396428
Operating Cash Flow Rank
Current Ratio
0.06789993133047025
Current Ratio Rank
Total Avg Rank
Total Avg Percentile


------------------
 1 BRSP
8.61 11.4
Ticker
BRSP
Price
8.61
Projected Price Growth
32.40418118466901
Market Cap
1116140032
PEG
1.520247618333332
PEG Rank
PE
8.524753
PE Rank
5 Year EPS Growth
14.473684210526313
EPS Growth Rank
Increasing EPS Score 0-4
1
EPS Surprise Avg
-109.04874999999998
ROE
-6.06
ROE Rank
Operating Cash Flow
-0.005128942135790853
Operating Cash Flow Rank
Current Ratio
37.02670186686781
Current Ratio Rank
Total Avg Rank
Total Avg Percentile


------------------
 2 EML
------------------
 3 ALJJ
------------

2.51 18.14
Ticker
PASG
Price
2.51
Projected Price Growth
622.7091633466136
Market Cap
136312320
PEG
0.030208350223880596
PEG Rank
PE
-0.7254335
PE Rank
5 Year EPS Growth
12.98701298701297
EPS Growth Rank
Increasing EPS Score 0-4
0
EPS Surprise Avg
-20.244999999999997
ROE
-59.48
ROE Rank
Operating Cash Flow
-3.483868310497268
Operating Cash Flow Rank
Current Ratio
11.019933554817277
Current Ratio Rank
Total Avg Rank
Total Avg Percentile


------------------
 32 RPT
13.8 14.89
Ticker
RPT
Price
13.8
Projected Price Growth
7.89855072463768
Market Cap
1195825280
PEG
13.8
PEG Rank
PE
690.0
PE Rank
5 Year EPS Growth
11.84210526315786
EPS Growth Rank
Increasing EPS Score 0-4
2
EPS Surprise Avg
333.06050000000005
ROE
8.28
ROE Rank
Operating Cash Flow
0.09225282503414876
Operating Cash Flow Rank
Current Ratio
0.9961256150790407
Current Ratio Rank
Total Avg Rank
Total Avg Percentile


------------------
 33 METC
14.72 18.65
Ticker
METC
Price
14.72
Projected Price Growth
26.698369565217373
Market 

32.74 28.29
------------------
 62 GSKY
nan 8.25
------------------
 63 XOM
85.13 92.47
------------------
 64 WIZP
------------------
 65 ZMTP
------------------
 66 ELMD
12.5 21.0
------------------
 67 NBSE
1.44 15.5
------------------
 68 CGIX
------------------
 69 BGSF
12.15 19.75
Ticker
BGSF
Price
12.15
Projected Price Growth
62.5514403292181
Market Cap
126666176
PEG
0.6658241181818183
PEG Rank
PE
11.35514
PE Rank
5 Year EPS Growth
107.6923076923077
EPS Growth Rank
Increasing EPS Score 0-4
2
EPS Surprise Avg
-6.923499999999997
ROE
14.72
ROE Rank
Operating Cash Flow
0.09293289251862666
Operating Cash Flow Rank
Current Ratio
2.119887435394258
Current Ratio Rank
Total Avg Rank
Total Avg Percentile


------------------
 70 PEYE
------------------
 71 NPTN
15.23 15.5
Ticker
NPTN
Price
15.23
Projected Price Growth
1.7728168089297411
Market Cap
809142464
PEG
1.2100105932203389
PEG Rank
PE
95.1875
PE Rank
5 Year EPS Growth
-54.347826086956516
EPS Growth Rank
Increasing EPS Score 0-4
3
E

------------------
 103 PROG
------------------
 104 BKR
32.2 39.4
Ticker
BKR
Price
32.2
Projected Price Growth
22.360248447204953
Market Cap
34901581824
PEG
0.836788624642857
PEG Rank
PE
26.178862
PE Rank
5 Year EPS Growth
266.66666666666674
EPS Growth Rank
Increasing EPS Score 0-4
3
EPS Surprise Avg
-57.471500000000006
ROE
-1.89
ROE Rank
Operating Cash Flow
0.1278957008943002
Operating Cash Flow Rank
Current Ratio
1.6504163014899211
Current Ratio Rank
Total Avg Rank
Total Avg Percentile


------------------
 105 LPX
63.73 84.63
Ticker
LPX
Price
63.73
Projected Price Growth
32.79460222815001
Market Cap
5482551808
PEG
-0.0513352372972973
PEG Rank
PE
4.998431
PE Rank
5 Year EPS Growth
492.30769230769243
EPS Growth Rank
Increasing EPS Score 0-4
3
EPS Surprise Avg
5.448
ROE
110.59
ROE Rank
Operating Cash Flow
1.5539267015706806
Operating Cash Flow Rank
Current Ratio
2.5356125356125356
Current Ratio Rank
Total Avg Rank
Total Avg Percentile


------------------
 106 NSIT
101.78 120.33
Ticke

Ticker
ROK
Price
260.53
Projected Price Growth
23.0645223198864
Market Cap
30272542720
PEG
2.263782959069766
PEG Rank
PE
24.417057
PE Rank
5 Year EPS Growth
32.04022988505749
EPS Growth Rank
Increasing EPS Score 0-4
3
EPS Surprise Avg
9.0135
ROE
42.42
ROE Rank
Operating Cash Flow
0.15747736497034032
Operating Cash Flow Rank
Current Ratio
1.0237952008555578
Current Ratio Rank
Total Avg Rank
Total Avg Percentile


------------------
 143 ACER
2.19 10.0
Ticker
ACER
Price
2.19
Projected Price Growth
356.62100456621005
Market Cap
33529338
PEG
-0.0006399232743362831
PEG Rank
PE
-1.033019
PE Rank
5 Year EPS Growth
-77.37789203084833
EPS Growth Rank
Increasing EPS Score 0-4
3
EPS Surprise Avg
-152.8695
ROE
-472.99
ROE Rank
Operating Cash Flow
-0.0035520543722424706
Operating Cash Flow Rank
Current Ratio
0.9649146082353285
Current Ratio Rank
Total Avg Rank
Total Avg Percentile


------------------
 144 WMPN
------------------
 145 MXL
44.14 74.1
Ticker
MXL
Price
44.14
Projected Price Growth
67.

...

6.17 11.5
------------------
 1707 GLSI
------------------
 1708 GFOO
------------------
 1709 QEGY
------------------
 1710 MLSS
1.08 4.0
------------------
 1711 HAYW
16.17 22.63
Ticker
HAYW
Price
16.17
Projected Price Growth
39.95052566481136
Market Cap
3708185088
PEG
4.1688281249999966
PEG Rank
PE
12.6328125
PE Rank
5 Year EPS Growth
0.0
EPS Growth Rank
Increasing EPS Score 0-4
0
EPS Surprise Avg
18.435
ROE
25.81
ROE Rank
Operating Cash Flow
0.11770793128716305
Operating Cash Flow Rank
Current Ratio
2.4699240449674855
Current Ratio Rank
Total Avg Rank
Total Avg Percentile


------------------
 1712 GTHX
6.38 41.5
------------------
 1713 TIG
4.7 6.75
------------------
 1714 WLMS
1.65 5.0
------------------
 1715 MCG
7.48 13.0
------------------
 1716 EBC
20.65 24.8
------------------
 1717 NREF
22.92 23.0
Ticker
NREF
Price
22.92
Projected Price Growth
0.3490401396160484
Market Cap
621182400
PEG
-0.3699058187
PEG Rank
PE
8.8494215
PE Rank
5 Year EPS Growth
50.95541401273888
EPS Gro

4.78 26.31
------------------
 1763 YAYO
------------------
 1764 ABSI
------------------
 1765 CBB
------------------
 1766 RSLS
0.96 4.75
------------------
 1767 LBBB
------------------
 1768 KIRK
7.63 21.0
------------------
 1769 USAU
5.95 20.75
------------------
 1770 UPH
0.91 4.13
------------------
 1771 IEC
------------------
 1772 IMUX
8.1 nan
------------------
 1773 MUX
------------------
 1774 ESGR
247.2 nan
------------------
 1775 MRSN
3.85 21.0
------------------
 1776 CVGI
7.36 15.5
------------------
 1777 SMAR
49.05 65.14
Ticker
SMAR
Price
49.05
Projected Price Growth
32.80326197757392
Market Cap
6309939200
PEG
1.3005681499999997
PEG Rank
PE
-74.31818
PE Rank
5 Year EPS Growth
-19.44444444444443
EPS Growth Rank
Increasing EPS Score 0-4
2
EPS Surprise Avg
43.463750000000005
ROE
-33.17
ROE Rank
Operating Cash Flow
-0.007051458378927544
Operating Cash Flow Rank
Current Ratio
1.4507211291174313
Current Ratio Rank
Total Avg Rank
Total Avg Percentile


------------------


------------------
 1822 VEL
10.24 15.25
Ticker
VEL
Price
10.24
Projected Price Growth
48.92578125
Market Cap
331618304
PEG
0.46987873333333324
PEG Rank
PE
9.061947
PE Rank
5 Year EPS Growth
11.49425287356323
EPS Growth Rank
Increasing EPS Score 0-4
1
EPS Surprise Avg
21.973749999999995
ROE
8.94
ROE Rank
Operating Cash Flow
0.02334776344131333
Operating Cash Flow Rank
Current Ratio
7.0516642743282905
Current Ratio Rank
Total Avg Rank
Total Avg Percentile


------------------
 1823 RSVR
8.95 13.33
------------------
 1824 FVIV
------------------
 1825 SVBL
------------------
 1826 PACQ
------------------
 1827 FULC
10.98 34.25
------------------
 1828 BBI
0.32 2.67
------------------
 1829 CLMT
14.0 25.2
------------------
 1830 FREE
7.14 16.38
------------------
 1831 WRTC
------------------
 1832 AAWW
68.42 102.25
Ticker
AAWW
Price
68.42
Projected Price Growth
49.44460684010523
Market Cap
1940637440
PEG
-0.3766714523950614
PEG Rank
PE
4.5674233
PE Rank
5 Year EPS Growth
276.3747454175

8.71 10.0
Ticker
SXC
Price
8.71
Projected Price Growth
14.81056257175659
Market Cap
725718976
PEG
-1.3385497466666654
PEG Rank
PE
11.311688
PE Rank
5 Year EPS Growth
-1375.0000000000005
EPS Growth Rank
Increasing EPS Score 0-4
2
EPS Surprise Avg
169.46099999999998
ROE
9.42
ROE Rank
Operating Cash Flow
0.21583333333333332
Operating Cash Flow Rank
Current Ratio
1.4923161361141604
Current Ratio Rank
Total Avg Rank
Total Avg Percentile


------------------
 1873 TTOO
0.43 1.05
------------------
 1874 NVTR
------------------
 1875 SPFI
25.24 33.5
Ticker
SPFI
Price
25.24
Projected Price Growth
32.725832012678296
Market Cap
446334048
PEG
1.368498015000002
PEG Rank
PE
10.64979
PE Rank
5 Year EPS Growth
87.57396449704137
EPS Growth Rank
Increasing EPS Score 0-4
2
EPS Surprise Avg
22.353333333333335
ROE
15.08
ROE Rank
Operating Cash Flow
0.027549859376126792
Operating Cash Flow Rank
Current Ratio
0.17364853243818684
Current Ratio Rank
Total Avg Rank
Total Avg Percentile


------------------
 18

0.61 4.5
------------------
 1916 FCBC
27.38 32.0
------------------
 1917 DRH
10.43 10.7
Ticker
DRH
Price
10.43
Projected Price Growth
2.588686481303927
Market Cap
2224134912
PEG
1.0456140631578945
PEG Rank
PE
49.666668
PE Rank
5 Year EPS Growth
-315.5555555555556
EPS Growth Rank
Increasing EPS Score 0-4
2
EPS Surprise Avg
61.981500000000004
ROE
-12.06
ROE Rank
Operating Cash Flow
-0.0015972671292533208
Operating Cash Flow Rank
Current Ratio
4.238073458349076
Current Ratio Rank
Total Avg Rank
Total Avg Percentile


------------------
 1918 INST
17.63 31.0
------------------
 1919 PYCR
24.76 34.77
------------------
 1920 PLPC
------------------
 1921 RAIL
5.18 6.25
Ticker
RAIL
Price
5.18
Projected Price Growth
20.65637065637066
Market Cap
85601048
PEG
-0.1869786020454545
PEG Rank
PE
-30.470587
PE Rank
5 Year EPS Growth
11.016949152542363
EPS Growth Rank
Increasing EPS Score 0-4
2
EPS Surprise Avg
-35.665
ROE
-287.42
ROE Rank
Operating Cash Flow
-0.27380881771451165
Operating Cash Flow

In [93]:
rankedDF

,Ticker,Price,Projected Price Growth,Market Cap,PEG,PEG Rank,PE,PE Rank,5 Year EPS Growth,EPS Growth Rank,Increasing EPS Score 0-4,EPS Surprise Avg,ROE,ROE Rank,Operating Cash Flow Ratio,Operating Cash Flow Ratio Rank,Current Ratio,Current Ratio Rank,Total Avg Rank,Total Avg Percentile
0,VLY,12.43,27.755430,6294403072,0.833904,NaN,10.272727,NaN,75.757576,NaN,4,-1.174500,9.79,NaN,0.021822,NaN,0.067900,NaN,0,0
1,BRSP,8.61,32.404181,1116140032,1.520248,NaN,8.524753,NaN,14.473684,NaN,1,-109.048750,-6.06,NaN,-0.005129,NaN,37.026702,NaN,0,0
2,KELYA,21.23,39.755064,836364352,0.569417,NaN,12.712575,NaN,-31.363636,NaN,2,68.225000,12.30,NaN,0.054557,NaN,1.450603,NaN,0,0
3,ROAD,26.42,24.261923,1388238848,1.428913,NaN,37.211270,NaN,-64.210526,NaN,0,109.105000,4.39,NaN,0.121945,NaN,1.899344,NaN,0,0
4,R,67.46,37.222058,3448609024,-0.437874,NaN,5.815517,NaN,111.504425,NaN,2,16.588500,20.64,NaN,0.197175,NaN,0.768735,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
769,SBH,15.94,41.154329,1754611456,1.058273,NaN,6.202334,NaN,30.729167,NaN,2,12.411500,132.41,NaN,0.148793,NaN,2.080936,NaN,0,0
770,TSC,31.77,3.084671,1066877888,0.603012,NaN,14.915492,NaN,43.548387,NaN,3,16.057000,9.80,NaN,0.008690,NaN,0.054479,NaN,0,0
771,TMST,21.65,11.778291,1004163776,-0.044739,NaN,5.835579,NaN,-426.262626,NaN,2,34.800000,29.18,NaN,0.398341,NaN,2.321770,NaN,0,0
772,BWB,16.57,22.691611,467559008,1.076362,NaN,10.165644,NaN,69.230769,NaN,2,3.078125,14.17,NaN,0.017505,NaN,0.055692,NaN,0,0


In [131]:
rankedDF.to_csv('topPerformingInsiderStocks2019-2022.csv', index=False)

import csv
print(len(skipped))

with open('skippedForecast-less_Stocks(small cap).csv', 'w') as f: 
    write = csv.writer(f) 
    write.writerow(skipped)

1157


In [102]:
#use the next 5 years measure to get fair value
yf.get_analysts_info('kelya')['Growth Estimates']

,Growth Estimates,KELYA,Industry,Sector(s),S&P 500
0,Current Qtr.,108.30%,NaN,NaN,NaN
1,Next Qtr.,-30.60%,NaN,NaN,NaN
2,Current Year,10.60%,NaN,NaN,NaN
3,Next Year,28.70%,NaN,NaN,NaN
4,Next 5 Years (per annum),15.00%,NaN,NaN,NaN
5,Past 5 Years (per annum),26.02%,NaN,NaN,NaN


In [36]:
# read in newRankedDF instead of running yahoo_fin calls block
newRankedDF = pd.read_csv('topPerformingInsiderStocks2019-2022.csv', skiprows = 0)

# newRankedDF = rankedDF
# newRankedDF['PEG Rank'] = rankedDF['PEG'].rank(ascending=True)
# newRankedDF['PE Rank'] = rankedDF['PE'].rank(ascending=True)
# newRankedDF['EPS Growth Rank'] = rankedDF['5 Year EPS Growth'].rank(ascending=False)
# # newRankedDF['Increasing EPS Rank'] = rankedDF['Increasing EPS Score 0-4'].rank(method='max')
# newRankedDF['ROE Rank'] = rankedDF['ROE'].rank(ascending=False)
# newRankedDF['Operating Cash Flow Ratio Rank'] = rankedDF['Operating Cash Flow Ratio'].rank(ascending=False)
# newRankedDF['Current Ratio Rank'] = rankedDF['Current Ratio'].rank(ascending=False)

newRankedDF['Buy/Sell Freq'] = 0.0
newRankedDF['Avg Buy Price'] = 0.0
newRankedDF['Avg Buy/Current'] = 0.0
newRankedDF['Score'] = 0

# newRankedDF = newRankedDF.drop(columns=['Total Avg Percentile', 'Increasing EPS Rank', 'PE Rank', 'PEG Rank', 'EPS Growth Rank', 'ROE Rank', 'Operating Cash Flow Ratio Rank', 'Current Ratio Rank', 'Total Avg Rank'])


# newRankedDF['Total Avg Rank'] = newRankedDF[['PEG Rank', 'PE Rank', 'EPS Growth Rank', 'ROE Rank', 'Operating Cash Flow Ratio Rank', 'Current Ratio Rank', 'Increasing EPS Rank']].mean(axis=1)
# newRankedDF['Total Avg Rank'] = newRankedDF[['PEG Rank', 'PE Rank', 'EPS Growth Rank', 'ROE Rank', 'Operating Cash Flow Ratio Rank', 'Current Ratio Rank']].mean(axis=1)
# newRankedDF = newRankedDF.sort_values(by=['Total Avg Rank'])


#FILTERING
newRankedDF = newRankedDF.loc[newRankedDF['Increasing EPS Score 0-4'] >= 3]
newRankedDF = newRankedDF.loc[newRankedDF['ROE'] > .15]
newRankedDF = newRankedDF.loc[newRankedDF['Operating Cash Flow Ratio'] >= .2]
newRankedDF = newRankedDF.loc[newRankedDF['Current Ratio'] >= 1]
# newRankedDF = newRankedDF.loc[(newRankedDF['PEG'] <= 2.5) & (newRankedDF['PEG'] > 0)]
newRankedDF = newRankedDF.loc[newRankedDF['Projected Price Growth'] > 5]
newRankedDF = newRankedDF.loc[newRankedDF['5 Year EPS Growth'] > 40]

#USE THE SCORE COLUMN---THAT WAS THE MAIN POINT OF USING PYTHON FOR THIS

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.display.float_format = "{:,.2f}".format

newRankedDF


,Ticker,Price,Projected Price Growth,Market Cap,PEG,PE,5 Year EPS Growth,Increasing EPS Score 0-4,EPS Surprise Avg,ROE,Operating Cash Flow Ratio,Current Ratio,Buy/Sell Freq,Avg Buy Price,Avg Buy/Current,Score
19,METC,14.72,26.70,651704448,-0.02,2.32,264.00,3,170.05,20.92,0.45,1.86,0.00,0.00,0.00,0
23,FANG,130.10,32.51,23081693184,-0.36,5.82,117.57,3,53.24,19.75,0.41,1.01,0.00,0.00,0.00,0
33,EVR,107.34,27.76,4395487232,1.23,8.56,222.98,3,39.17,55.59,0.64,1.07,0.00,0.00,0.00,0
37,SNDR,22.74,23.79,4039942912,-0.99,8.99,143.62,3,7.83,18.10,0.37,1.81,0.00,0.00,0.00,0
48,LSCC,47.29,62.83,6505354752,1.91,33.54,"1,070.00",4,27.64,24.10,0.53,2.84,0.00,0.00,0.00,0
49,AKAM,117.14,15.65,18847942656,1.92,19.79,119.08,4,6.74,14.84,0.39,2.43,0.00,0.00,0.00,0
51,LPX,63.73,32.79,5482551808,-0.05,5.00,492.31,3,5.45,110.59,1.55,2.54,0.00,0.00,0.00,0
59,HIBB,44.96,60.88,583230080,0.45,4.57,561.96,4,34.62,51.08,0.39,1.42,0.00,0.00,0.00,0
123,WGO,55.34,36.30,1813851520,-0.20,4.48,327.82,4,21.86,32.18,0.24,2.60,0.00,0.00,0.00,0
124,DFIN,29.63,40.63,964726080,-14.74,7.32,226.32,3,49.72,46.70,0.36,1.07,0.00,0.00,0.00,0


### What if for testing: you buy every stock that the GCs did and at that same point and take the difference between price then and price now. 

Also, get percentage of insider transactions the past year that were buys. Additionally,  consider the price they bought/sold at.

In [37]:
newRankedDF['Skept Buyers Ratio'] = -1.0
newRankedDF['Top Performers to Buyers Ratio'] = -1.0
newRankedDF['# of Buyers'] = -1
newRankedDF

,Ticker,Price,Projected Price Growth,Market Cap,PEG,PE,5 Year EPS Growth,Increasing EPS Score 0-4,EPS Surprise Avg,ROE,Operating Cash Flow Ratio,Current Ratio,Buy/Sell Freq,Avg Buy Price,Avg Buy/Current,Score,Skept Buyers Ratio,Top Performers to Buyers Ratio,# of Buyers
19,METC,14.72,26.70,651704448,-0.02,2.32,264.00,3,170.05,20.92,0.45,1.86,0.00,0.00,0.00,0,-1.00,-1.00,-1
23,FANG,130.10,32.51,23081693184,-0.36,5.82,117.57,3,53.24,19.75,0.41,1.01,0.00,0.00,0.00,0,-1.00,-1.00,-1
33,EVR,107.34,27.76,4395487232,1.23,8.56,222.98,3,39.17,55.59,0.64,1.07,0.00,0.00,0.00,0,-1.00,-1.00,-1
37,SNDR,22.74,23.79,4039942912,-0.99,8.99,143.62,3,7.83,18.10,0.37,1.81,0.00,0.00,0.00,0,-1.00,-1.00,-1
48,LSCC,47.29,62.83,6505354752,1.91,33.54,"1,070.00",4,27.64,24.10,0.53,2.84,0.00,0.00,0.00,0,-1.00,-1.00,-1
49,AKAM,117.14,15.65,18847942656,1.92,19.79,119.08,4,6.74,14.84,0.39,2.43,0.00,0.00,0.00,0,-1.00,-1.00,-1
51,LPX,63.73,32.79,5482551808,-0.05,5.00,492.31,3,5.45,110.59,1.55,2.54,0.00,0.00,0.00,0,-1.00,-1.00,-1
59,HIBB,44.96,60.88,583230080,0.45,4.57,561.96,4,34.62,51.08,0.39,1.42,0.00,0.00,0.00,0,-1.00,-1.00,-1
123,WGO,55.34,36.30,1813851520,-0.20,4.48,327.82,4,21.86,32.18,0.24,2.60,0.00,0.00,0.00,0,-1.00,-1.00,-1
124,DFIN,29.63,40.63,964726080,-14.74,7.32,226.32,3,49.72,46.70,0.36,1.07,0.00,0.00,0.00,0,-1.00,-1.00,-1


In [40]:
topPerformers = ["Sr.VP", "Chairman", "VP GC", "Mktg", "Engineering", "Chief People Officer", "Corporate Development", "Business Dev", "Corp Secretary", "Co-Founder", "Global Sales", "Corp. Sec.", "VP Sales", "Interim CEO", "Principal Accounting Officer", "Admin.", "Member of Group > 10%", "Chief Business Officer", "SVP-GC", "CRO", "Retail", "VP Finance", "CSO", "Division Pres", "HR", "Pres of Subsidiary", "EVP - Operations", "Sec", "Gen. Counsel", "Sr. Vice Pres", "Corporate VP", "PAO", "North America", "Exec VP", "Sales", "GM", "C.E.O.", "Corp. Dev.", "Secy.", "Asia", "Chief Scientific Officer", "International", "Marketing", "CMO", "Exec.VP", "Ex. VP", "Chief Operations Officer", "CHRO", "Chief Revenue Officer", "See remarks", "13(d) group member - >10%", "VP of Finance", "SVP Finance", "VP Operations", "Secy", "D", "VP of Marketing", "Exec Officer", "VP Corporate Development", "Acting CFO", "Vice-Pres", "Chief Risk Officer", "VP HR", "Research", "Operations", "VP of Operations", "SVP of Sales", "Controller", "MD", "CFO", "See Remarks", "E.VP", "EVP - CFO", "GC", "VP of Sales", "Chief Admin Officer", "Worldwide Sales", "IT", "Treasurer", "Sec'y", "Strategy", "Chief Development Officer", "VP", "CTO", "Sec.", "Exec COB", "Chief Administrative Officer", "VP - Finance", "Senior Vice-Pres", "SVP", "Chief Accounting Officer", "CIO", "Senior VP", "Sr EVP", "VP-Finance", "EVP Finance", "IR"]


#function that takes in ticker and all-trades-dataframe and returns
# avg price bought at
# buy-to-sell ratio
# skeptical buyers ratio to total buyers
# number of "top performers" purchases ratio to total buyers
# total # of buyers
for index, line in newRankedDF.iterrows():
    
    ticker = line['Ticker']
    
    buyCount = 0
    sellCount = 0

    sellers = []
    buyers = []
    sellThenBuyers = 0
    sellThenBuyersList = []
    topPerformersCount = 0
    buySum = 0

    for ind, row in (data.loc[data['Ticker'] == ticker]).iterrows():
        if row['Trade Type'][0] == 'P':
            buyCount +=1
            buySum += row['Price']
            if (row['Insider Name'] in sellers) and (row['Insider Name'] not in sellThenBuyersList):
                sellThenBuyers += 1
                sellThenBuyersList.append(row['Insider Name'])
                
                
                titles = row['Title'].split(', ')
                
                for title in titles:
                    if title in topPerformers:
                        topPerformersCount += 1

            if row['Insider Name'] not in buyers:
                buyers.append(row['Insider Name'])

        elif row['Trade Type'][0] == 'S':
            sellCount += 1

            if row['Insider Name'] not in sellers:
                sellers.append(row['Insider Name'])

    buyAvg = buySum/buyCount
    
    if sellCount > 0:
        buyToSellRatio = buyCount/sellCount
    else:
        buyToSellRatio = 1
    skeptBuyersRatio = sellThenBuyers / len(buyers)
    buyersCount = len(buyers)
    topPerfRatio = topPerformersCount / len(buyers)

#     print(ticker, 'buyAvg:', buyAvg, 'buyToSellRatio:', buyToSellRatio, 'skeptBuyersRatio:', skeptBuyersRatio, 'buyersCount:', buyersCount, 'topPerfRatio:', topPerfRatio)
    
    newRankedDF.at[index, 'Avg Buy Price'] = buyAvg
    newRankedDF.at[index, 'Buy/Sell Freq'] = buyToSellRatio
    newRankedDF.at[index, 'Avg Buy/Current'] = buyAvg/line['Price']  #we want this number to be as high as possible
    newRankedDF.at[index, 'Skept Buyers Ratio'] = skeptBuyersRatio
    newRankedDF.at[index, 'Top Performers to Buyers Ratio'] = topPerfRatio   #actually a ratio!!!
    newRankedDF.at[index, '# of Buyers'] = buyersCount

METC buyAvg: 3.9249999999999994 buyToSellRatio: 1.2 skeptBuyersRatio: 0.0 buyersCount: 4 topPerfRatio: 0.0
FANG buyAvg: 56.644000000000005 buyToSellRatio: 0.11363636363636363 skeptBuyersRatio: 0.3333333333333333 buyersCount: 3 topPerfRatio: 0.3333333333333333
EVR buyAvg: 42.0925 buyToSellRatio: 0.19047619047619047 skeptBuyersRatio: 0.5 buyersCount: 2 topPerfRatio: 0.0
SNDR buyAvg: 21.405 buyToSellRatio: 0.1111111111111111 skeptBuyersRatio: 0.0 buyersCount: 2 topPerfRatio: 0.0
LSCC buyAvg: 12.315000000000001 buyToSellRatio: 0.01015228426395939 skeptBuyersRatio: 0.0 buyersCount: 2 topPerfRatio: 0.0
AKAM buyAvg: 111.04 buyToSellRatio: 0.010526315789473684 skeptBuyersRatio: 1.0 buyersCount: 1 topPerfRatio: 1.0
LPX buyAvg: 41.21857142857143 buyToSellRatio: 0.4666666666666667 skeptBuyersRatio: 0.0 buyersCount: 3 topPerfRatio: 0.0
HIBB buyAvg: 44.27642857142856 buyToSellRatio: 0.6666666666666666 skeptBuyersRatio: 0.25 buyersCount: 8 topPerfRatio: 0.0
WGO buyAvg: 36.596666666666664 buyToSellRa

In [41]:
newRankedDF

,Ticker,Price,Projected Price Growth,Market Cap,PEG,PE,5 Year EPS Growth,Increasing EPS Score 0-4,EPS Surprise Avg,ROE,Operating Cash Flow Ratio,Current Ratio,Buy/Sell Freq,Avg Buy Price,Avg Buy/Current,Score,Skept Buyers Ratio,Top Performers to Buyers Ratio,# of Buyers
19,METC,14.72,26.70,651704448,-0.02,2.32,264.00,3,170.05,20.92,0.45,1.86,1.20,3.92,0.27,0,0.00,0.00,4
23,FANG,130.10,32.51,23081693184,-0.36,5.82,117.57,3,53.24,19.75,0.41,1.01,0.11,56.64,0.44,0,0.33,0.33,3
33,EVR,107.34,27.76,4395487232,1.23,8.56,222.98,3,39.17,55.59,0.64,1.07,0.19,42.09,0.39,0,0.50,0.00,2
37,SNDR,22.74,23.79,4039942912,-0.99,8.99,143.62,3,7.83,18.10,0.37,1.81,0.11,21.41,0.94,0,0.00,0.00,2
48,LSCC,47.29,62.83,6505354752,1.91,33.54,"1,070.00",4,27.64,24.10,0.53,2.84,0.01,12.32,0.26,0,0.00,0.00,2
49,AKAM,117.14,15.65,18847942656,1.92,19.79,119.08,4,6.74,14.84,0.39,2.43,0.01,111.04,0.95,0,1.00,1.00,1
51,LPX,63.73,32.79,5482551808,-0.05,5.00,492.31,3,5.45,110.59,1.55,2.54,0.47,41.22,0.65,0,0.00,0.00,3
59,HIBB,44.96,60.88,583230080,0.45,4.57,561.96,4,34.62,51.08,0.39,1.42,0.67,44.28,0.98,0,0.25,0.00,8
123,WGO,55.34,36.30,1813851520,-0.20,4.48,327.82,4,21.86,32.18,0.24,2.60,0.38,36.60,0.66,0,0.20,0.20,5
124,DFIN,29.63,40.63,964726080,-14.74,7.32,226.32,3,49.72,46.70,0.36,1.07,1.43,9.73,0.33,0,0.00,0.00,8


#### Now we filter by insider data

In [42]:
insiderFilteredDF = newRankedDF

insiderFilteredDF = insiderFilteredDF.loc[insiderFilteredDF['Buy/Sell Freq'] >= .3]
insiderFilteredDF = insiderFilteredDF.loc[insiderFilteredDF['Avg Buy/Current'] <= .7]
#^ if is is selling at a very discounted price relative to recent buys, do insiders even have that much credibility here?
insiderFilteredDF = insiderFilteredDF.loc[insiderFilteredDF['# of Buyers'] >= 2]

insiderFilteredDF

,Ticker,Price,Projected Price Growth,Market Cap,PEG,PE,5 Year EPS Growth,Increasing EPS Score 0-4,EPS Surprise Avg,ROE,Operating Cash Flow Ratio,Current Ratio,Buy/Sell Freq,Avg Buy Price,Avg Buy/Current,Score,Skept Buyers Ratio,Top Performers to Buyers Ratio,# of Buyers
19,METC,14.72,26.70,651704448,-0.02,2.32,264.00,3,170.05,20.92,0.45,1.86,1.20,3.92,0.27,0,0.00,0.00,4
51,LPX,63.73,32.79,5482551808,-0.05,5.00,492.31,3,5.45,110.59,1.55,2.54,0.47,41.22,0.65,0,0.00,0.00,3
123,WGO,55.34,36.30,1813851520,-0.20,4.48,327.82,4,21.86,32.18,0.24,2.60,0.38,36.60,0.66,0,0.20,0.20,5
124,DFIN,29.63,40.63,964726080,-14.74,7.32,226.32,3,49.72,46.70,0.36,1.07,1.43,9.73,0.33,0,0.00,0.00,8
236,WY,41.52,7.03,31019468800,-0.25,15.04,193.04,3,108.59,26.74,0.46,3.29,0.67,25.30,0.61,0,0.00,0.00,5
452,RFP,13.91,52.62,1068240704,-0.04,2.39,"4,907.69",3,-60.29,23.78,0.32,2.29,0.65,6.85,0.49,0,0.00,0.00,8
712,WLKP,27.01,68.46,951140928,2.20,12.22,42.68,3,-0.44,42.52,0.80,2.63,1.00,16.44,0.61,0,0.00,0.00,8


In [43]:
# TRY KEEPING ALL 800+ STOCKS IN DF AND THEN SCORING EACH ONE INSTEAD OF FILTERING THEM OUT. THEN SORT BY SCORE

In [45]:
#CHECK ASSUMPTIONS LISTED IN NOTES!!! ALSO, QUESTION EACH METRIC BEING TRACKED AND 1) IT'S IMPORTANCE, AND 2) IT'S POTENTIAL REDUNDANCY OF OTHER METRICS
